# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date
0,Kota,IN,25.18,75.83,83.08,73,100,9.31,1594688651
1,Upernavik,GL,72.79,-56.15,40.55,89,99,17.43,1594688234
2,Ushuaia,AR,-54.80,-68.30,33.80,80,40,19.46,1594687978
3,Adelaide,AU,-34.93,138.60,48.99,93,0,1.12,1594688440
4,Bengkulu,ID,-3.80,102.27,77.05,84,100,3.62,1594688251


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#store lat and long in locations
locations = cities_df[["Latitude", "Longitude"]]
weight = cities_df["Humidity"]

In [4]:
# Plot Heatmap

figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)

heat_layer.opacity = 0.30
# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#weather conditions
#max temp between 68 and 83
temp_cities = cities_df.loc[(cities_df["Temperature (F)"] > 68.0) & (cities_df["Temperature (F)"] < 83.0)]
#humidity <60%
humidity_cities = temp_cities.loc[temp_cities["Humidity"] < 60]
#cloudiness <20%
cloudy_cities = humidity_cities.loc[humidity_cities["Cloudiness"] == 0]
cloudy_cities
#windspeed betwen 5-10 mph
vacation_cities = cloudy_cities.loc[(cloudy_cities["Windspeed (mph)"] > 5.0) & (cloudy_cities["Windspeed (mph)"] < 10.0)]
vacation_cities

,City,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date
5,Kumul,CN,42.80,93.45,77.16,28,0,5.73,1594688113
48,Aswān,EG,24.09,32.91,81.95,21,0,9.37,1594688045
90,Pánormos,GR,35.42,24.69,78.01,52,0,9.55,1594688664
104,Daşoguz,TM,41.84,59.97,75.20,44,0,6.93,1594688666
111,Albany,US,42.60,-73.97,78.01,57,0,6.60,1594688219
142,Lensk,RU,60.73,114.93,77.25,40,0,5.19,1594688671
259,Jalu,LY,29.03,21.55,76.19,50,0,8.25,1594688164
311,Abhā,SA,18.22,42.51,71.60,52,0,7.45,1594688446
461,Aljezur,PT,37.32,-8.80,81.00,54,0,8.10,1594688716
546,Ubari,LY,26.59,12.78,76.84,28,0,8.79,1594688441


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities.copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date,Hotel Name
5,Kumul,CN,42.80,93.45,77.16,28,0,5.73,1594688113,
48,Aswān,EG,24.09,32.91,81.95,21,0,9.37,1594688045,
90,Pánormos,GR,35.42,24.69,78.01,52,0,9.55,1594688664,
104,Daşoguz,TM,41.84,59.97,75.20,44,0,6.93,1594688666,
111,Albany,US,42.60,-73.97,78.01,57,0,6.60,1594688219,
142,Lensk,RU,60.73,114.93,77.25,40,0,5.19,1594688671,
259,Jalu,LY,29.03,21.55,76.19,50,0,8.25,1594688164,
311,Abhā,SA,18.22,42.51,71.60,52,0,7.45,1594688446,
461,Aljezur,PT,37.32,-8.80,81.00,54,0,8.10,1594688716,
546,Ubari,LY,26.59,12.78,76.84,28,0,8.79,1594688441,


In [11]:
#search for hotels within 5000 meters
#set parameters
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}
#loop through  hotel_df
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #pull out list of latitude and longitude
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    #add location to parameters
    params["location"] = f"{lat}, {lon}"
    
    #make api request
    hotel_name = requests.get(base_url, params=params)
    
    #get json
    hotel_name = hotel_name.json()
    
    #add results to the df
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Windspeed (mph),Date,Hotel Name
5,Kumul,CN,42.80,93.45,77.16,28,0,5.73,1594688113,Hami Rubik's Cube Hotel
48,Aswān,EG,24.09,32.91,81.95,21,0,9.37,1594688045,Sofitel Legend Old Cataract Aswan
90,Pánormos,GR,35.42,24.69,78.01,52,0,9.55,1594688664,Grecotel Creta Palace
104,Daşoguz,TM,41.84,59.97,75.20,44,0,6.93,1594688666,Hotel UMAR
111,Albany,US,42.60,-73.97,78.01,57,0,6.60,1594688219,Renaissance Albany Hotel
142,Lensk,RU,60.73,114.93,77.25,40,0,5.19,1594688671,Mini-hotel Uyut
259,Jalu,LY,29.03,21.55,76.19,50,0,8.25,1594688164,Awjilah Tourist Hotel
311,Abhā,SA,18.22,42.51,71.60,52,0,7.45,1594688446,Abha Palace Hotel
461,Aljezur,PT,37.32,-8.80,81.00,54,0,8.10,1594688716,Hotel Oriental
546,Ubari,LY,26.59,12.78,76.84,28,0,8.79,1594688441,فندق أوباري


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…